In [86]:
# GOOGLE COLAB ONLY: Install Pyomo and Ipopt
# !pip install pyomo
# !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
# !unzip -o -q ipopt-linux64

from pyomo.environ import (
    SolverFactory,
    AbstractModel,
    Param,
    Set,
    Var,
    Objective,
    Constraint,
)
from pyomo.core import NonNegativeIntegers, PositiveReals, Any, NonNegativeReals  # type: ignore

solver = SolverFactory("ipopt")

## Storage Problem/System Summary

$$
\begin{aligned}
  &\text{min} \sum_{t=0}^T w(t) \\
  &\text{s.t.} \\
  &t = \{1, 2, ..., T\} \text{ as time points} \\
  &\text{$s(t)$ as reservoir storage, $a(t)$ as surface area, $r(t)$ as rainfall, } \\
  &\text{$e(t)$ as evaporation, $q(t)$ as pumping amount, $w(t)$ as overflow at or over time $t$} \\
  &s(t + 1) = s(t) + r(t) - e(t)- q(t) - w(t), \forall t \\
  &\text{$s(t) \leq \bar S$ for maximum capacity, $a(t) \leq \bar A$ as maximum surface area, and $q(t) \leq \bar Q$ as maximum pumping rate} \\
  &r(t) = A_c p(t) \text{ where $A_c$ is a constant catchment area and $p(t)$ is precipitation} \\
  &e(t) = a(t) \cdot e_0(t) \text{ where $e_0(t)$ is the evaporation coefficient at time $t$} \\
  &s(t) = c_1 a(t) ^ {c_2} \text{(power law relationship with constants)}
\end{aligned}
$$


In [95]:
model = AbstractModel()

# Indices input parameters
model.T_intervals = Set(domain=NonNegativeIntegers)  # Time intervals
model.T_points = Set(
    domain=NonNegativeIntegers
)  # Time points, TODO: assert that this is [...T_intervals, 1]

# Constant input parameters
model.c1 = Param(within=PositiveReals, mutable=False)  # Scalar constant c1
model.c2 = Param(within=PositiveReals, mutable=False)  # Scalar constant c2
model.A_c = Param(
    within=PositiveReals, mutable=False
)  # Catchment area in square meters
model.S_max = Param(within=PositiveReals, mutable=False)  # Maximum storage capacity
model.A_max = Param(within=PositiveReals, mutable=False)  # Maximum free surface area
model.Q_max = Param(
    within=NonNegativeReals, mutable=False
)  # Upper limit for pumping rate

# Time-varying input parameters
model.P = Param(model.T_intervals, within=Any)  # Precipitation
model.E_coeff = Param(model.T_intervals, within=Any)  # Evaporation coefficients

# Calculated Parameters
model.R = Param(  # Rainfall
    model.T_intervals,
    default=lambda model, t: model.A_c * model.P[t],
    within=NonNegativeReals,
)

# Optimizable Variables
model.S = Var(model.T_points, domain=NonNegativeReals)  # Storage
model.A = Var(model.T_points, domain=NonNegativeReals)  # Surface area

model.E = Var(model.T_intervals, domain=NonNegativeReals)  # Evaporation
model.Q = Var(model.T_intervals, domain=NonNegativeReals)  # Pumping
model.W = Var(model.T_intervals, domain=NonNegativeReals)  # Overflow

# Objective
model.min_overflow = Objective(
    expr=lambda model: sum(model.W[t] for t in model.T_intervals)
)

# Constraints
model.upper_S = Constraint(
    model.T_points, rule=lambda model, t: model.S[t] <= model.S_max
)

model.upper_A = Constraint(
    model.T_points, rule=lambda model, t: model.A[t] <= model.A_max
)

model.upper_Q = Constraint(
    model.T_intervals, rule=lambda model, t: model.Q[t] <= model.Q_max
)

model.evaporation = Constraint(
    model.T_intervals,
    rule=lambda model, t: model.E[t] == model.E_coeff[t] ** model.A[t],
)

model.storage = Constraint(
    model.T_points, rule=lambda model, t: model.S[t] == model.c1 * model.A[t] + model.c2
)

model.dynamics = Constraint(
    model.T_intervals,
    rule=lambda model, t: model.S[t + 1]
    == model.S[t] + model.R[t] - model.E[t] - model.Q[t] - model.W[t],
)


In [96]:
num_intervals = 12
interval_indices = list(range(num_intervals))
points_indices = list(range(num_intervals + 1))

data = {
    None: {
        "T_intervals": {None: interval_indices},
        "T_points": {None: points_indices},
        "c1": {None: 0.5},
        "c2": {None: 0.5},
        "A_c": {None: 100},
        "S_max": {None: 100},
        "A_max": {None: 100},
        "Q_max": {None: 100},
        "P": {i: 0.5 for i in interval_indices},
        "E_coeff": {i: 0.7 for i in interval_indices},
    }
}

instance = model.create_instance(data)
results = solver.solve(instance)
instance.pprint()


2 Set Declarations
    T_intervals : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain              : Size : Members
        None :     1 : NonNegativeIntegers :   12 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}
    T_points : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain              : Size : Members
        None :     1 : NonNegativeIntegers :   13 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

9 Param Declarations
    A_c : Size=1, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Key  : Value
        None :   100
    A_max : Size=1, Index=None, Domain=PositiveReals, Default=None, Mutable=False
        Key  : Value
        None :   100
    E_coeff : Size=12, Index=T_intervals, Domain=Any, Default=None, Mutable=False
        Key : Value
          0 :   0.7
          1 :   0.7
          2 :   0.7
          3 :   0.7
          4 :   0.7
          5 :   0.7
          6 :   0.7
          7 :   0.7
          8 :   0.7
          9 :   